# 검색을 위한 위키피디아 문서 임베딩

이 노트북은 [Question_answer_using_embeddings.ipynb](질문_답변_사용_embeddings.ipynb)에 사용되는 검색용 위키백과 문서 데이터 세트를 어떻게 준비했는지 보여줍니다.

절차:

0. 전제 조건: 라이브러리 가져오기, API 키 설정(필요한 경우)
1. 수집합니다: 2022년 올림픽에 대한 수백 개의 위키피디아 기사를 다운로드합니다.
2. 청크: 문서를 반독립적인 짧은 섹션으로 분할하여 임베드합니다.
3. 임베드: 각 섹션은 OpenAI API로 임베드됩니다.
4. 저장: 임베딩은 CSV 파일에 저장됩니다(대규모 데이터 세트의 경우 벡터 데이터베이스 사용).

## 0. 전제 조건

### 라이브러리 가져오기

In [1]:
# imports
import mwclient  # for downloading example Wikipedia articles
import mwparserfromhell  # for splitting Wikipedia articles into sections
import openai  # for generating embeddings
import pandas as pd  # for DataFrames to store article sections and embeddings
import re  # for cutting <ref> links out of Wikipedia articles
import tiktoken  # for counting tokens


터미널에서 `pip install`으로 누락된 라이브러리를 설치합니다. 예,

```zsh
pip install openai
```

(노트북 셀에서 '!pip install openai'를 사용하여 이 작업을 수행할 수도 있습니다.)

라이브러리를 설치한 경우 노트북 커널을 다시 시작해야 합니다.

API 키 설정(필요한 경우) ###

OpenAI 라이브러리는 `OPENAI_API_KEY` 환경 변수에서 API 키를 읽으려고 시도합니다. 아직 설정하지 않았다면 [이 지침](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety)에 따라 이 환경 변수를 설정하세요.

## 1. 문서 수집

이 예에서는 2022년 동계 올림픽과 관련된 수백 개의 Wikipedia 문서를 다운로드하겠습니다.

In [2]:
# get Wikipedia pages about the 2022 Winter Olympics

CATEGORY_TITLE = "Category:2022 Winter Olympics"
WIKI_SITE = "en.wikipedia.org"


def titles_from_category(
    category: mwclient.listing.Category, max_depth: int
) -> set[str]:
    """Return a set of page titles in a given Wiki category and its subcategories."""
    titles = set()
    for cm in category.members():
        if type(cm) == mwclient.page.Page:
            # ^type() used instead of isinstance() to catch match w/ no inheritance
            titles.add(cm.name)
        elif isinstance(cm, mwclient.listing.Category) and max_depth > 0:
            deeper_titles = titles_from_category(cm, max_depth=max_depth - 1)
            titles.update(deeper_titles)
    return titles


site = mwclient.Site(WIKI_SITE)
category_page = site.pages[CATEGORY_TITLE]
titles = titles_from_category(category_page, max_depth=1)
# ^note: max_depth=1 means we go one level deep in the category tree
print(f"Found {len(titles)} article titles in {CATEGORY_TITLE}.")


Found 731 article titles in Category:2022 Winter Olympics.


## 2. 청크 문서

이제 참조 문서가 생겼으니 검색할 수 있도록 준비해야 합니다.

GPT는 한 번에 제한된 양의 텍스트만 읽을 수 있으므로 각 문서를 읽을 수 있을 정도로 짧은 청크로 분할합니다.

위키백과 문서를 예로 들어 보겠습니다:
- 외부 링크 및 각주와 같이 관련성이 떨어지는 섹션을 삭제합니다.
- 참조 태그(예: <ref>), 공백, 매우 짧은 섹션을 제거하여 텍스트를 정리합니다.
- 각 문서를 섹션으로 나누기
- 각 섹션의 텍스트에 제목과 부제를 추가하여 GPT가 문맥을 이해하는 데 도움이 되도록 합니다.
- 섹션이 긴 경우(예: 1,600토큰 이상) 문단처럼 의미적 경계를 따라 재귀적으로 더 작은 섹션으로 분할합니다.

In [3]:
# define functions to split Wikipedia pages into sections

SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]


def all_subsections_from_section(
    section: mwparserfromhell.wikicode.Wikicode,
    parent_titles: list[str],
    sections_to_ignore: set[str],
) -> list[tuple[list[str], str]]:
    """
    From a Wikipedia section, return a flattened list of all nested subsections.
    Each subsection is a tuple, where:
        - the first element is a list of parent subtitles, starting with the page title
        - the second element is the text of the subsection (but not any children)
    """
    headings = [str(h) for h in section.filter_headings()]
    title = headings[0]
    if title.strip("=" + " ") in sections_to_ignore:
        # ^wiki headings are wrapped like "== Heading =="
        return []
    titles = parent_titles + [title]
    full_text = str(section)
    section_text = full_text.split(title)[1]
    if len(headings) == 1:
        return [(titles, section_text)]
    else:
        first_subtitle = headings[1]
        section_text = section_text.split(first_subtitle)[0]
        results = [(titles, section_text)]
        for subsection in section.get_sections(levels=[len(titles) + 1]):
            results.extend(all_subsections_from_section(subsection, titles, sections_to_ignore))
        return results


def all_subsections_from_title(
    title: str,
    sections_to_ignore: set[str] = SECTIONS_TO_IGNORE,
    site_name: str = WIKI_SITE,
) -> list[tuple[list[str], str]]:
    """From a Wikipedia page title, return a flattened list of all nested subsections.
    Each subsection is a tuple, where:
        - the first element is a list of parent subtitles, starting with the page title
        - the second element is the text of the subsection (but not any children)
    """
    site = mwclient.Site(site_name)
    page = site.pages[title]
    text = page.text()
    parsed_text = mwparserfromhell.parse(text)
    headings = [str(h) for h in parsed_text.filter_headings()]
    if headings:
        summary_text = str(parsed_text).split(headings[0])[0]
    else:
        summary_text = str(parsed_text)
    results = [([title], summary_text)]
    for subsection in parsed_text.get_sections(levels=[2]):
        results.extend(all_subsections_from_section(subsection, [title], sections_to_ignore))
    return results


In [4]:
# split pages into sections
# may take ~1 minute per 100 articles
wikipedia_sections = []
for title in titles:
    wikipedia_sections.extend(all_subsections_from_title(title))
print(f"Found {len(wikipedia_sections)} sections in {len(titles)} pages.")


Found 5730 sections in 731 pages.


In [5]:
# clean text
def clean_section(section: tuple[list[str], str]) -> tuple[list[str], str]:
    """
    Return a cleaned up section with:
        - <ref>xyz</ref> patterns removed
        - leading/trailing whitespace removed
    """
    titles, text = section
    text = re.sub(r"<ref.*?</ref>", "", text)
    text = text.strip()
    return (titles, text)


wikipedia_sections = [clean_section(ws) for ws in wikipedia_sections]

# filter out short/blank sections
def keep_section(section: tuple[list[str], str]) -> bool:
    """Return True if the section should be kept, False otherwise."""
    titles, text = section
    if len(text) < 16:
        return False
    else:
        return True


original_num_sections = len(wikipedia_sections)
wikipedia_sections = [ws for ws in wikipedia_sections if keep_section(ws)]
print(f"Filtered out {original_num_sections-len(wikipedia_sections)} sections, leaving {len(wikipedia_sections)} sections.")


Filtered out 530 sections, leaving 5200 sections.


In [6]:
# print example data
for ws in wikipedia_sections[:5]:
    print(ws[0])
    display(ws[1][:77] + "...")
    print()


['Lviv bid for the 2022 Winter Olympics']


'{{Olympic bid|2022|Winter|\n| Paralympics = yes\n| logo = Lviv 2022 Winter Olym...'


['Lviv bid for the 2022 Winter Olympics', '==History==']


'[[Image:Lwów - Rynek 01.JPG|thumb|right|200px|View of Rynok Square in Lviv]]\n...'


['Lviv bid for the 2022 Winter Olympics', '==Venues==']


'{{Location map+\n|Ukraine\n|border =\n|caption = Venue areas\n|float = left\n|widt...'


['Lviv bid for the 2022 Winter Olympics', '==Venues==', '===City zone===']


'The main Olympic Park would be centered around the [[Arena Lviv]], hosting th...'


['Lviv bid for the 2022 Winter Olympics', '==Venues==', '===Mountain zone===', '====Venue cluster Tysovets-Panasivka====']


'An existing military ski training facility in [[Tysovets, Skole Raion|Tysovet...'

다음으로 긴 섹션을 재귀적으로 작은 섹션으로 분할해 보겠습니다.

텍스트를 섹션으로 분할하는 완벽한 방법은 없습니다.

몇 가지 장단점이 있습니다:
- 더 많은 컨텍스트가 필요한 질문에는 긴 섹션이 더 좋을 수 있습니다.
- 섹션이 길면 여러 주제가 뒤섞여 검색에 좋지 않을 수 있습니다.
- 섹션이 짧으면 토큰 수에 비례하는 비용을 줄이는 데 더 좋습니다.
- 섹션이 짧을수록 더 많은 섹션을 검색할 수 있으므로 리콜에 도움이 될 수 있습니다.
- 섹션이 겹치면 섹션 경계로 인해 답이 잘리는 것을 방지하는 데 도움이 될 수 있습니다.

여기서는 간단한 접근 방식을 사용하여 섹션을 각각 1,600개 토큰으로 제한하고 너무 긴 섹션은 재귀적으로 절반으로 줄이겠습니다. 유용한 문장이 중간에 잘리지 않도록 가능한 경우 단락 경계를 따라 분할할 것입니다.

In [7]:
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use


def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str, str]:
    """Split a string in two, on a delimiter, trying to balance tokens on each side."""
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # no delimiter found
    elif len(chunks) == 2:
        return chunks  # no need to search for halfway point
    else:
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        best_diff = halfway
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        return [left, right]


def truncated_string(
    string: str,
    model: str,
    max_tokens: int,
    print_warning: bool = True,
) -> str:
    """Truncate a string to a maximum number of tokens."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Warning: Truncated string from {len(encoded_string)} tokens to {max_tokens} tokens.")
    return truncated_string


def split_strings_from_subsection(
    subsection: tuple[list[str], str],
    max_tokens: int = 1000,
    model: str = GPT_MODEL,
    max_recursion: int = 5,
) -> list[str]:
    """
    Split a subsection into a list of subsections, each with no more than max_tokens.
    Each subsection is a tuple of parent titles [H1, H2, ...] and text (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # if length is fine, return string
    if num_tokens_in_string <= max_tokens:
        return [string]
    # if recursion hasn't found a split after X iterations, just truncate
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # otherwise, split in half and recurse
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]:
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # if either half is empty, retry with a more fine-grained delimiter
                continue
            else:
                # recurse on each half
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1,
                    )
                    results.extend(half_strings)
                return results
    # otherwise no split was found, so just truncate (should be very rare)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]


In [8]:
# split sections into chunks
MAX_TOKENS = 1600
wikipedia_strings = []
for section in wikipedia_sections:
    wikipedia_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wikipedia_sections)} Wikipedia sections split into {len(wikipedia_strings)} strings.")


5200 Wikipedia sections split into 6059 strings.


In [9]:
# print example data
print(wikipedia_strings[1])


Lviv bid for the 2022 Winter Olympics

==History==

[[Image:Lwów - Rynek 01.JPG|thumb|right|200px|View of Rynok Square in Lviv]]

On 27 May 2010, [[President of Ukraine]] [[Viktor Yanukovych]] stated during a visit to [[Lviv]] that Ukraine "will start working on the official nomination of our country as the holder of the Winter Olympic Games in [[Carpathian Mountains|Carpathians]]".

In September 2012, [[government of Ukraine]] approved a document about the technical-economic substantiation of the national project "Olympic Hope 2022". This was announced by Vladyslav Kaskiv, the head of Ukraine´s Derzhinvestproekt (State investment project). The organizers announced on their website venue plans featuring Lviv as the host city and location for the "ice sport" venues, [[Volovets]] (around {{convert|185|km|mi|abbr=on}} from Lviv) as venue for the [[Alpine skiing]] competitions and [[Tysovets, Skole Raion|Tysovets]] (around {{convert|130|km|mi|abbr=on}} from Lviv) as venue for all other "sn

## 3. 문서 청크 임베드

이제 라이브러리를 더 짧은 독립된 문자열로 분할했으므로 각각에 대한 임베딩을 계산할 수 있습니다.

(대규모 임베딩 작업의 경우, 속도 제한을 벗어나지 않도록 스로틀링하면서 요청을 병렬화하는 [api_request_parallel_processor.py](api_request_parallel_processor.py) 같은 스크립트를 사용하세요.)

In [10]:
# calculate embeddings
EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI's best embeddings as of Apr 2023
BATCH_SIZE = 1000  # you can submit up to 2048 embedding inputs per request

embeddings = []
for batch_start in range(0, len(wikipedia_strings), BATCH_SIZE):
    batch_end = batch_start + BATCH_SIZE
    batch = wikipedia_strings[batch_start:batch_end]
    print(f"Batch {batch_start} to {batch_end-1}")
    response = openai.Embedding.create(model=EMBEDDING_MODEL, input=batch)
    for i, be in enumerate(response["data"]):
        assert i == be["index"]  # double check embeddings are in same order as input
    batch_embeddings = [e["embedding"] for e in response["data"]]
    embeddings.extend(batch_embeddings)

df = pd.DataFrame({"text": wikipedia_strings, "embedding": embeddings})


Batch 0 to 999
Batch 1000 to 1999
Batch 2000 to 2999
Batch 3000 to 3999
Batch 4000 to 4999
Batch 5000 to 5999
Batch 6000 to 6999


## 4. 문서 청크 및 임베딩 저장

이 예제에서는 몇 천 개의 문자열만 사용하므로 CSV 파일에 저장합니다.

(더 큰 데이터 세트의 경우 벡터 데이터베이스를 사용하는 것이 더 효율적입니다.)

In [11]:
# save document chunks and embeddings

SAVE_PATH = "data/winter_olympics_2022.csv"

df.to_csv(SAVE_PATH, index=False)
